In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

np.random.seed(42)
n = 5000

df = pd.DataFrame({
    '지점': np.random.choice(['서울', '부산', '대전'], n),
    '장소': np.random.choice(['실내', '실외'], n),
    '상태': np.random.choice(['정상', '주의', '위험'], n),
    '장비명': np.random.choice(['A1', 'B2', 'C3'], n),
    '온도': np.random.normal(25, 5, n),
    '진동': np.random.normal(0.5, 0.2, n),
    '전류': np.random.normal(3.0, 0.5, n),
    '압력': np.random.normal(1.0, 0.1, n),
    '시작시간': pd.date_range('2024-01-01', periods=n, freq='H')
})
df['종료시간'] = df['시작시간'] + pd.Timedelta(hours=1)
df['에너지사용량'] = df['온도']*0.5 + df['전류']*1.2 + np.random.normal(0, 0.5, n)

C:\Users\user\AppData\Local\Temp\ipykernel_8172\2996910454.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  '시작시간': pd.date_range('2024-01-01', periods=n, freq='H')


In [12]:
df

,지점,장소,상태,장비명,온도,진동,전류,압력,시작시간,종료시간,에너지사용량
0,대전,실내,주의,B2,24.118187,0.383603,2.439801,1.086833,2024-01-01 00:00:00,2024-01-01 01:00:00,15.853302
1,서울,실내,위험,A1,17.365554,0.411542,2.698062,0.784365,2024-01-01 01:00:00,2024-01-01 02:00:00,11.701820
2,대전,실내,위험,A1,23.684701,0.682598,3.510641,0.965160,2024-01-01 02:00:00,2024-01-01 03:00:00,16.111446
3,대전,실외,정상,C3,24.218521,0.447242,3.115501,1.085519,2024-01-01 03:00:00,2024-01-01 04:00:00,15.356875
4,서울,실외,위험,A1,16.508494,0.518519,3.180125,0.813375,2024-01-01 04:00:00,2024-01-01 05:00:00,12.118880
...,...,...,...,...,...,...,...,...,...,...,...
4995,서울,실내,주의,A1,24.905006,0.543409,3.604103,0.741683,2024-07-27 03:00:00,2024-07-27 04:00:00,16.374590
4996,부산,실외,위험,C3,24.252701,0.503841,2.802598,1.038421,2024-07-27 04:00:00,2024-07-27 05:00:00,15.041399
4997,대전,실내,정상,B2,19.819118,0.441876,3.850387,0.976770,2024-07-27 05:00:00,2024-07-27 06:00:00,15.273830
4998,부산,실내,주의,B2,27.725716,0.359282,3.382686,1.075058,2024-07-27 06:00:00,2024-07-27 07:00:00,18.514228


In [4]:
#토치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#시퀀스
def create_seq(data,seq_len,max_n): 
    xs,ns,ys = [],[],[]
    #시계열에서 시퀀스를 생성할 때 슬라이딩 윈도우 방식 사용
    #len(data) 전체 시계열 데이터수 , seq_len 모델에 들어갈 과거 시간 길이 ex:10시간  max_n 예측할 최대 미래시간 
    for i in range(len(data)-seq_len-max_n):  # - seq_len - max_n의미 i가 10시간 만큼 가져오고 i:i+seq_len인데 뒤에 seq_len+max_n만큼 여유 주는것
        x = data[i:i+seq_len,:-1] #i부터 i+seq_len만큼만 데이터 즉 피쳐값만 가져오고 마지막 열은 :-1임으로 안가져옴
        for n in range(1,max_n+1):#가져온 x의 데이터를 여러 시간차 의 타켓y를 만듬
            y = data[i+seq_len+n-1,-1] #n시간뒤 target값
            xs.append(x)
            ns.append([n])
            ys.append(y)
    return np.array(xs),np.array(ns),np.array(ys)
#데이터셋구성
class TimeSeriesDataset(Dataset):
    def __init__(self,x,n_array,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.n = torch.tensor(n_array,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index],self.n[index],self.y[index]
#모델구성
class LSTMwithN(nn.Module):
    def __init__(self,input_dim,hidden_dim,num_layer):
        # input_dim 입력 피쳐 수  hidden_dim LSTM 내부 뉴런 수 num_layers LSTM 층 수
        # batch_first=True	입력 shape을 (batch, seq_len, input_dim)으로 설정
        # fc = nn.Linear(hidden_dim*2, 1)	마지막 hidden state와 n_feat을 합쳐서 1개의 출력값(에너지 예측)으로 변환
        super().__init__()
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layer,batch_first=True)
        self.fc_n = nn.Linear(1,hidden_dim) 
        self.fc = nn.Linear(hidden_dim*2,1)

    def forward(self,x,n):
        # x: (batch, seq_len, input_dim)
        # n: (batch, 1)
        #out, (h_n, c_n) = self.lstm(x)
        #out =  모든 시점의 hidden state	(batch, seq_len, hidden_dim) 
        # hidden_dim = LSTM이 각 시점에서 기억하고 있는 정보(메모리)의 요약본
        #h_n	마지막 시점 hidden state (각 레이어별)	(num_layers, batch, hidden_dim)
        #c_n	마지막 시점 cell state (각 레이어별)	(num_layers, batch, hidden_dim)
        out,_ = self.lstm(x) #모든 시점의 LSTM 출력 → shape: (batch, seq_len, hidden_dim)
        last_out = out[:,-1,:] #out[:, -1, :] → 64개 시퀀스 중에서 각자의 10번째 시점의 hidden state 
        n_feat = self.fc_n(n)
        combined = torch.cat([last_out,n_feat],dim=1)
        return self.fc(combined).squeeze()
#문자열 처리
for col in ['지점','장소','상태','장비명']:
    df[col] = LabelEncoder().fit_transform(df[col])
#피처선택
features = ['지점','장소','상태','장비명','온도','진동','전류','압력']
target = '에너지사용량'
#정규화
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
x_raw = df[features].values
y_raw = df[target].values
x_scaled = scaler_x.fit_transform(x_raw)
y_scaled = scaler_y.fit_transform(y_raw.reshape(-1, 1))
data_scaled = np.hstack([x_scaled,y_scaled])
#데이터셋 구성
seq_len = 10
max_n = 6
X,n_array,y=create_seq(data_scaled,seq_len,max_n)
dataset = TimeSeriesDataset(X,n_array,y)
loader = DataLoader(dataset,batch_size=64,shuffle=True)
#학습구성
num_epochs = 100
model = LSTMwithN(input_dim=X.shape[2],hidden_dim=64,num_layer=2).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for xb,nb,yb in loader:
        xb,nb,yb = xb.to(device), nb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb,nb)
        loss = loss_fn(pred,yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch +1}  Loss : {total_loss/len(loader):.7f}')


Epoch 1  Loss : 0.0232500
Epoch 2  Loss : 0.0210533
Epoch 3  Loss : 0.0213479
Epoch 4  Loss : 0.0213769
Epoch 5  Loss : 0.0215242
Epoch 6  Loss : 0.0213120
Epoch 7  Loss : 0.0214143
Epoch 8  Loss : 0.0213986
Epoch 9  Loss : 0.0211330
Epoch 10  Loss : 0.0212745
Epoch 11  Loss : 0.0213690
Epoch 12  Loss : 0.0214261
Epoch 13  Loss : 0.0211004
Epoch 14  Loss : 0.0211413
Epoch 15  Loss : 0.0213493
Epoch 16  Loss : 0.0212451
Epoch 17  Loss : 0.0210256
Epoch 18  Loss : 0.0211236
Epoch 19  Loss : 0.0211823
Epoch 20  Loss : 0.0212058
Epoch 21  Loss : 0.0211502
Epoch 22  Loss : 0.0212601
Epoch 23  Loss : 0.0209568
Epoch 24  Loss : 0.0209480
Epoch 25  Loss : 0.0210985
Epoch 26  Loss : 0.0207630
Epoch 27  Loss : 0.0207318
Epoch 28  Loss : 0.0208403
Epoch 29  Loss : 0.0207735
Epoch 30  Loss : 0.0206962
Epoch 31  Loss : 0.0203866
Epoch 32  Loss : 0.0201822
Epoch 33  Loss : 0.0202190
Epoch 34  Loss : 0.0200964
Epoch 35  Loss : 0.0200582
Epoch 36  Loss : 0.0202684
Epoch 37  Loss : 0.0200595
Epoch 38  

In [10]:
#예측구성
# 최근 10개 시점 
past_seq = data_scaled[-seq_len:,:-1]
past_seq_tensor = torch.tensor(past_seq,dtype=torch.float32).unsqueeze(0).to(device)
n=2
n_tnesor = torch.tensor([[n]],dtype=torch.float32).to(device)
model.eval()
with torch.no_grad():
    pred = model(past_seq_tensor,n_tnesor).cpu().item()
#역정규화
real_pred = scaler_y.inverse_transform([[pred]])[0][0]
print(f"{n}시간 뒤 예측값: {real_pred:.4f}")

2시간 뒤 예측값: 17.0715


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight

from collections import defaultdict
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 1. 샘플 더미 데이터 생성
np.random.seed(42)
n = 5000

지점 = np.random.choice(['서울', '부산', '대전'], n)
장소 = np.random.choice(['실내', '실외'], n)
상태 = np.random.choice(['정상', '주의', '위험'], n)
장비명 = np.random.choice(['A1', 'B2', 'C3'], n)
온도 = np.random.normal(25, 5, n)
진동 = np.random.normal(0.5, 0.2, n)
전류 = np.random.normal(3.0, 0.5, n)
압력 = np.random.normal(1.0, 0.1, n)
시작시간 = pd.to_datetime('2024-01-01') + pd.to_timedelta(np.random.randint(0, 60*24*30, size=n), unit='m')
종료시간 = 시작시간 + pd.to_timedelta(np.random.randint(30, 180, size=n), unit='m')  # 30분~3시간

에너지사용량 = 온도 * 0.5 + 전류 * 1.2 + np.random.normal(0, 0.5, n)
장비상태 = np.random.choice(['정상', '경고', '위험'], n)

df = pd.DataFrame({
    '지점': 지점,
    '장소': 장소,
    '상태': 상태,
    '장비명': 장비명,
    '온도': 온도,
    '진동': 진동,
    '전류': 전류,
    '압력': 압력,
    '시작시간': 시작시간,
    '종료시간': 종료시간,
    '에너지사용량': 에너지사용량,
    '장비상태': 장비상태
})

df = df.sort_values(['지점', '시작시간']).reset_index(drop=True)
df['시간차'] = (df['종료시간'] - df['시작시간']).dt.total_seconds() / 3600.0

# 2단계 전처리 및 시퀀스 생성
#범주형 컬럼 인코딩 
cat_cols = ['지점','장소','상태','장비명']
label_encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
# 타켓 인코딩 (분류용)
target_cls_le = LabelEncoder()
df['장비상태_코드'] = target_cls_le.fit_transform(df['장비상태'])
#정규화(입력 features, 회귀타겟 따로)
features = ['지점','장소','상태','장비명','온도','진동','전류','압력','시간차']
target_reg = '에너지사용량'
target_cls = '장비상태_코드'
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
x_scaled = scaler_x.fit_transform(df[features])
y_scaled = scaler_y.fit_transform(df[[target_reg]])
#데이터 통합
data_scaled = np.hstack([x_scaled,y_scaled,df[[target_cls]].values])
#지점별로 시퀀스 분리
seq_len = 10 
max_n = 7*24 # 최대 예측 n (1주일)

def create_seq(df, data_scaled, seq_len,max_n):
    xs , ns, y_regs, y_clss = [],[],[],[] #	입력 시퀀스, n시간 뒤, 회귀 타겟, 분류 타겟 저장용 리스트
    for site in df['지점'].unique():
        site_idx = (df['지점'] == site)
        site_data = data_scaled[site_idx.to_numpy()] #스케일된 데이터 중 해당 지점 데이터 추출
        for i in range(len(site_data)-seq_len-max_n):
            seq_x = site_data[i:i+seq_len,:-2]
            for n in range(1, max_n+1):
                if i +seq_len + n >= len(site_data): #	범위를 초과하는 경우 생략
                    continue
                xs.append(seq_x) #입력 시퀀스 저장
                ns.append([n/max_n]) # 정규화된 n
                y_regs.append(site_data[i+seq_len+n,-2]) #n시간 뒤 에너지 사용량 (회귀 타겟) 저장 에너지 소비량
                y_clss.append(site_data[i+seq_len+n,-1]) #n시간 뒤 장비 상태 (분류 타겟) 저장 분류타겟 장비상태 코드 
    return np.array(xs),np.array(ns),np.array(y_regs),np.array(y_clss)

# 3단계: LSTM 멀티태스크 모델 구성 (회귀 + 분류)
class LSTMwithMultiOutput(nn.Module):
    def __init__(self, input_dim,hidden_dim,num_layers,num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layers,batch_first=True)
        self.fc_n = nn.Linear(1,hidden_dim)
        self.dropout = nn.Dropout(0.1)
        self.fc_combied = nn.Linear(hidden_dim*2,hidden_dim)
        #출력레이어 : 회귀 + 분류
        self.out_reg = nn.Linear(hidden_dim,1)
        self.out_cls = nn.Linear(hidden_dim,num_classes)
        
    def forward(self,x,n):
        lstm_out,_ = self.lstm(x)
        last_out = lstm_out[:,-1,:]
        n_feat = self.fc_n(n)
        combined = torch.cat([last_out,n_feat],dim=1)
        combined = self.dropout(self.fc_combied(combined))
        reg_out = self.out_reg(combined).squeeze(1)
        cls_out = self.out_cls(combined)
        return reg_out, cls_out
    
class MultiTargetDataset(Dataset):
    def __init__(self,x,n,y_regs,y_clss):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.n = torch.tensor(n,dtype=torch.float32)
        self.y_reg = torch.tensor(y_regs,dtype=torch.float32)
        self.y_cls =torch.tensor(y_clss,dtype=torch.long)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx],self.n[idx],self.y_reg[idx],self.y_cls[idx]

#data
X,n_array,y_regs,y_clss = create_seq(df,data_scaled,seq_len,max_n)

#hyper parameter
input_dim = X.shape[2]
hidden_dim = 128
num_layers = 2
num_classes = len(np.unique(y_clss))
batch_size = 64
epochs = 50
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = LSTMwithMultiOutput(input_dim,hidden_dim,num_layers,num_classes).to(device)
dataset = MultiTargetDataset(X,n_array,y_regs,y_clss)
loader = DataLoader(dataset,batch_size=batch_size,shuffle=True)


unique_classes = np.unique(y_clss)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_clss)
weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)


loss_fn_reg = nn.MSELoss()
loss_fn_cls = nn.CrossEntropyLoss(weight=weights_tensor)

optimizer = optim.Adam(model.parameters(),lr=lr)


for epoch in range(epochs):
    model.train()
    total_loss, total_reg_loss, total_cls_loss = 0, 0, 0
    all_preds, all_labels = [], []

    for xb, nb, yb_reg, yb_cls in loader:
        xb, nb = xb.to(device), nb.to(device)
        yb_reg, yb_cls = yb_reg.to(device), yb_cls.to(device)

        pred_reg, pred_cls = model(xb, nb)  # 회귀, 분류 예측값
        # print(yb_reg.min().item(), yb_reg.max().item())  # 정규화된 범위인지
        # print(pred_reg.min().item(), pred_reg.max().item())  # 튀는 값 있는지
        # print(pred_reg.shape, yb_reg.shape)

        # 손실 계산
        loss_reg = loss_fn_reg(pred_reg, yb_reg)
        loss_cls = loss_fn_cls(pred_cls, yb_cls)
        loss = loss_reg + loss_cls

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_reg_loss += loss_reg.item()
        total_cls_loss += loss_cls.item()

        # 분류 정확도 계산
        all_preds.extend(pred_cls.argmax(dim=1).cpu().numpy())
        all_labels.extend(yb_cls.cpu().numpy())
        

        n_batches = len(loader)
    avg_loss = total_loss / n_batches
    avg_reg  = total_reg_loss  / n_batches
    avg_cls  = total_cls_loss  / n_batches
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')       # 다중 클래스용
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    print(f"[{epoch+1}] Loss: {total_loss:.4f} | Reg: {total_reg_loss:.4f} | Cls: {total_cls_loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f} | P: {precision:.4f} | R: {recall:.4f}")





In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, precision_score, recall_score

from collections import defaultdict
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
# 1. 샘플 더미 데이터 생성
np.random.seed(42)
n = 5000

지점 = np.random.choice(['서울', '부산', '대전'], n)
장소 = np.random.choice(['실내', '실외'], n)
상태 = np.random.choice(['정상', '주의', '위험'], n)
장비명 = np.random.choice(['A1', 'B2', 'C3'], n)
온도 = np.random.normal(25, 5, n)
진동 = np.random.normal(0.5, 0.2, n)
전류 = np.random.normal(3.0, 0.5, n)
압력 = np.random.normal(1.0, 0.1, n)
시작시간 = pd.to_datetime('2024-01-01') + pd.to_timedelta(np.random.randint(0, 60*24*30, size=n), unit='m')
종료시간 = 시작시간 + pd.to_timedelta(np.random.randint(30, 180, size=n), unit='m')  # 30분~3시간

에너지사용량 = 온도 * 0.5 + 전류 * 1.2 + np.random.normal(0, 0.5, n)
장비상태 = np.random.choice(['정상', '경고', '위험'], n)

df = pd.DataFrame({
    '지점': 지점,
    '장소': 장소,
    '상태': 상태,
    '장비명': 장비명,
    '온도': 온도,
    '진동': 진동,
    '전류': 전류,
    '압력': 압력,
    '시작시간': 시작시간,
    '종료시간': 종료시간,
    '에너지사용량': 에너지사용량,
    '장비상태': 장비상태
})

df = df.sort_values(['지점', '시작시간']).reset_index(drop=True)
df['시간차'] = (df['종료시간'] - df['시작시간']).dt.total_seconds() / 3600.0
df['장비상태'] = df.apply(lambda row: (
    '위험' if row['온도'] > 30 and row['진동'] > 0.8 else
    '경고' if row['전류'] > 3.5 or row['압력'] < 0.9 else
    '정상'), axis=1)

# 2단계 전처리 및 시퀀스 생성
#범주형 컬럼 인코딩 
cat_cols = ['지점','장소','상태','장비명']
label_encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
# 타켓 인코딩 (분류용)
target_cls_le = LabelEncoder()
df['장비상태_코드'] = target_cls_le.fit_transform(df['장비상태'])
#정규화(입력 features, 회귀타겟 따로)
features = ['지점','장소','상태','장비명','온도','진동','전류','압력','시간차']
target_reg = '에너지사용량'
target_cls = '장비상태_코드'
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
x_scaled = scaler_x.fit_transform(df[features])
y_scaled = scaler_y.fit_transform(df[[target_reg]])
#데이터 통합
# data_scaled = np.hstack([x_scaled,y_scaled,df[[target_cls]].values])
# data_scaled = np.hstack([x_scaled, y_scaled, df[[target_cls]].to_numpy()])  # ← 이건 OK
data_scaled = np.hstack([x_scaled, y_scaled, df[target_cls].to_numpy().reshape(-1, 1)])  # ← Fix: reshape to 2D
#지점별로 시퀀스 분리
seq_len = 10 
max_n = 7*24 # 최대 예측 n (1주일)

def create_seq(df, data_scaled, seq_len,max_n):
    xs , ns, y_regs, y_clss = [],[],[],[] #	입력 시퀀스, n시간 뒤, 회귀 타겟, 분류 타겟 저장용 리스트
    for site in df['지점'].unique():
        site_idx = (df['지점'] == site)
        site_data = data_scaled[site_idx.to_numpy()] #스케일된 데이터 중 해당 지점 데이터 추출
        for i in range(len(site_data)-seq_len-max_n):
            seq_x = site_data[i:i+seq_len,:-2]
            for n in range(1, max_n+1):
                if i +seq_len + n >= len(site_data): #	범위를 초과하는 경우 생략
                    continue
                xs.append(seq_x) #입력 시퀀스 저장
                ns.append([n/max_n]) # 정규화된 n
                y_regs.append(site_data[i+seq_len+n,-2]) #n시간 뒤 에너지 사용량 (회귀 타겟) 저장 에너지 소비량
                y_clss.append(site_data[i+seq_len+n,-1]) #n시간 뒤 장비 상태 (분류 타겟) 저장 분류타겟 장비상태 코드 
    return np.array(xs),np.array(ns),np.array(y_regs),np.array(y_clss)


class LSTMwithMultiOutput(nn.Module):
    def __init__(self, input_dim,hidden_dim,num_layers,num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        # n 피처용 MLP
        self.fc_n = nn.Sequential(
            nn.Linear(1, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim)
        )
        # 결합 후 MLP
        self.fc_combined = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.1)
        )
        self.out_reg = nn.Linear(hidden_dim, 1)
        self.out_cls = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, n):
        lstm_out, _ = self.lstm(x)
        last_out   = lstm_out[:, -1, :]
        n_feat     = self.fc_n(n)           # now non-linear
        comb       = torch.cat([last_out, n_feat], dim=1)
        comb       = self.fc_combined(comb) # now non-linear + BN
        return self.out_reg(comb).squeeze(1), self.out_cls(comb)


    
class MultiTargetDataset(Dataset):
    def __init__(self,x,n,y_regs,y_clss):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.n = torch.tensor(n,dtype=torch.float32)
        self.y_reg = torch.tensor(y_regs,dtype=torch.float32)
        self.y_cls =torch.tensor(y_clss,dtype=torch.long)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx],self.n[idx],self.y_reg[idx],self.y_cls[idx]

#data
X,n_array,y_regs,y_clss = create_seq(df,data_scaled,seq_len,max_n)

#hyper parameter
input_dim = X.shape[2]
hidden_dim = 128
num_layers = 2
num_classes = len(np.unique(y_clss))
batch_size = 64
epochs = 50
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = LSTMwithMultiOutput(input_dim,hidden_dim,num_layers,num_classes).to(device)
dataset = MultiTargetDataset(X,n_array,y_regs,y_clss)
loader = DataLoader(dataset,batch_size=batch_size,shuffle=True)


unique_classes = np.unique(y_clss)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_clss)
weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)


loss_fn_reg = nn.MSELoss()
loss_fn_cls = nn.CrossEntropyLoss(weight=weights_tensor)

optimizer = optim.Adam(model.parameters(),lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

for epoch in range(epochs):
    model.train()
    total_loss, total_reg_loss, total_cls_loss = 0, 0, 0
    all_preds, all_labels = [], []

    for xb, nb, yb_reg, yb_cls in loader:
        xb, nb = xb.to(device), nb.to(device)
        yb_reg, yb_cls = yb_reg.to(device), yb_cls.to(device)

        pred_reg, pred_cls = model(xb, nb)  # 회귀, 분류 예측값
        # print(yb_reg.min().item(), yb_reg.max().item())  # 정규화된 범위인지
        # print(pred_reg.min().item(), pred_reg.max().item())  # 튀는 값 있는지
        # print(pred_reg.shape, yb_reg.shape)

        # 손실 계산
        loss_reg = loss_fn_reg(pred_reg, yb_reg)
        loss_cls = loss_fn_cls(pred_cls, yb_cls)
        
        alpha = 1.0  # 회귀 손실 비중
        beta = 2.0   # 분류 손실 비중 (더 작게 시작)

        loss = alpha * loss_reg + beta * loss_cls

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)

        optimizer.step()

        total_loss += loss.item()
        total_reg_loss += loss_reg.item()
        total_cls_loss += loss_cls.item()

        # 분류 정확도 계산
        all_preds.extend(pred_cls.argmax(dim=1).cpu().numpy())
        all_labels.extend(yb_cls.cpu().numpy())
        

    n_batches = len(loader)
        
    scheduler.step()
    curr_lr = optimizer.param_groups[0]['lr']

    avg_loss = total_loss / n_batches
    avg_reg  = total_reg_loss  / n_batches
    avg_cls  = total_cls_loss  / n_batches
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')       # 다중 클래스용
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    print(f"[{epoch+1}] LR = {curr_lr:.5f} Loss: {avg_loss:.4f} | Reg: {avg_reg:.4f} | Cls: {avg_cls:.4f} | Acc: {acc:.4f} | F1: {f1:.4f} | precision: {precision:.4f} | recall: {recall:.4f}")